### Retrive table description

Parse the dbt schema file and retrive table description along with column details

### Create and store table descripton embeddings

### Generate the sql statement based on user prompt

Call a function that the table descriptions that will return the most likely tables to have this information based a neighboard search.

Feed back the table information to the prompt

Enginier the prompt in such a way that only SQL will be returned using a mult shoot strategy with examples

In [ ]:
functions = [
    {
        "name": "retrive_table",
        "description": """Retrieve table information based on the description provided by the user.
          The information will be returned in a json like this:
          {
            "name": "the name of the table",
            "description": "the description of the info contained in the table",
            "columns": [
              {
                "name": "column name",
                "description: "description of the data in the column",
                "properties": "properties of the column value, e.g. not_null, unique"
              }
            ]
          }
          
          ### EXAMPLE
          {
            "name": "stg_dosare",
            "description": "Informații despre dosarele din baza de date a instanțelor judecătorești",
            "columns": [
              {
                "name": "obiect",
                "description: "Obiectul judecății, acuzațiile",
                "properties": "not_null"
              }
            ]
          }
          """,
        "parametes": {
            "type": "type",
            "properties": {
                "description": {
                    "type": "string",
                    "description": "The description of the information contained in the table",
                }
            },
            "required": ["description"],
        },
    }
]

### Regression tests

Write a bunch of tests to figure out if we have regression in our assistant